# 🏆 Notebook Comparativo Integrador
## Selección del Mejor Modelo de Machine Learning

---

## 🎯 Objetivo

**Comparar todos los modelos entrenados** y seleccionar el mejor para el despliegue en producción.

## 📊 Modelos Evaluados

### Aprendizaje Supervisado - Regresión:
1. **Regresión Lineal**: Predicción de Valor FOB

### Aprendizaje Supervisado - Clasificación:
2. **Regresión Logística**: Clasificación multiclase
3. **K-Nearest Neighbors (KNN)**: Clasificación por vecinos
4. **Support Vector Machine (SVM)**: Márgenes óptimos
5. **Árboles de Decisión**: Reglas jerárquicas
6. **Naive Bayes**: Clasificación probabilística

### Aprendizaje No Supervisado:
7. **K-Means**: Clustering y segmentación

## 📏 Criterios de Evaluación

**Para Modelos de Regresión:**
- R² Score
- RMSE (Root Mean Squared Error)
- MAE (Mean Absolute Error)
- MAPE (Mean Absolute Percentage Error)

**Para Modelos de Clasificación:**
- Accuracy
- Precision
- Recall
- F1-Score
- Validación Cruzada

**Para Modelos de Clustering:**
- Silhouette Score
- Davies-Bouldin Index
- Inercia

## 1. Importación de Librerías

In [17]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

# Librerías de ML
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                            r2_score, mean_squared_error, mean_absolute_error)

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Librerías importadas exitosamente")

✅ Librerías importadas exitosamente


## 2. Carga de Modelos Entrenados

In [18]:
import os
import nbformat
print("📦 CARGANDO MODELOS DESDE NOTEBOOKS...")
print("="*80)

models = {}
notebooks_dir = os.path.join('..', '📓 NOTEBOOKS')
notebook_files = [
    ('Regresión Lineal', '02_Regresiòn_Lineal.ipynb', 'regresion'),
    ('Regresión Logística', '03_Regresiòn_Logistica.ipynb', 'clasificacion'),
    ('KNN', '04_KNN.ipynb', 'clasificacion'),
    ('SVM', '05_SVM.ipynb', 'clasificacion'),
    ('Árboles de Decisión', '06_Arboles_Decision.ipynb', 'clasificacion'),
    ('Naive Bayes', '07_Naive_Bayes.ipynb', 'clasificacion'),
    ('K-Means', '08_KMeans.ipynb', 'clustering')
]

def extraer_modelo_de_notebook(notebook_path):
    try:
        nb = nbformat.read(open(notebook_path, encoding='utf-8'), as_version=4)
        for cell in nb.cells:
            if cell.cell_type == 'code' and 'metrics' in cell.source:
                local_vars = {}
                exec(cell.source, {}, local_vars)
                if 'metrics' in local_vars:
                    return {'metrics': local_vars['metrics']}
        return None
    except Exception as e:
        print(f"   ⚠️ Error al leer {notebook_path}: {e}")
        return None

for name, filename, model_type in notebook_files:
    notebook_path = os.path.join(notebooks_dir, filename)
    if os.path.exists(notebook_path):
        modelo = extraer_modelo_de_notebook(notebook_path)
        if modelo:
            models[name] = {'package': modelo, 'type': model_type}
            print(f"   ✓ {name}: Métricas extraídas de {filename}")
        else:
            print(f"   ⚠️ {name}: No se encontraron métricas en {filename}")
            models[name] = {'package': None, 'type': model_type}
    else:
        print(f"   ⚠️ {name}: Notebook no encontrado - {filename}")
        models[name] = {'package': None, 'type': model_type}

print(f"\n📊 Total de modelos con métricas extraídas: {sum(1 for m in models.values() if m['package'] is not None)}")

def comparar_modelos_por_tipo(models, tipo):
    print(f'\n=== Comparativa de modelos: {tipo} ===')
    for nombre, info in models.items():
        if info['type'] == tipo and info['package'] is not None:
            print(f'- {nombre}: Métricas disponibles')
        elif info['type'] == tipo:
            print(f'- {nombre}: No disponible')

comparar_modelos_por_tipo(models, 'regresion')
comparar_modelos_por_tipo(models, 'clasificacion')
comparar_modelos_por_tipo(models, 'clustering')

📦 CARGANDO MODELOS DESDE NOTEBOOKS...
   ⚠️ Error al leer ..\📓 NOTEBOOKS\02_Regresiòn_Lineal.ipynb: invalid syntax (<string>, line 23)
   ⚠️ Regresión Lineal: No se encontraron métricas en 02_Regresiòn_Lineal.ipynb
✅ Librerías importadas
   ⚠️ Error al leer ..\📓 NOTEBOOKS\03_Regresiòn_Logistica.ipynb: name 'model_lr' is not defined
   ⚠️ Regresión Logística: No se encontraron métricas en 03_Regresiòn_Logistica.ipynb
   ⚠️ Error al leer ..\📓 NOTEBOOKS\04_KNN.ipynb: invalid syntax (<string>, line 20)
   ⚠️ KNN: No se encontraron métricas en 04_KNN.ipynb
   ⚠️ Error al leer ..\📓 NOTEBOOKS\05_SVM.ipynb: invalid syntax (<string>, line 20)
   ⚠️ SVM: No se encontraron métricas en 05_SVM.ipynb
   ⚠️ Error al leer ..\📓 NOTEBOOKS\06_Arboles_Decision.ipynb: invalid syntax (<string>, line 20)
   ⚠️ Árboles de Decisión: No se encontraron métricas en 06_Arboles_Decision.ipynb
   ⚠️ Error al leer ..\📓 NOTEBOOKS\07_Naive_Bayes.ipynb: invalid syntax (<string>, line 20)
   ⚠️ Naive Bayes: No se encontr

## 3. Extracción de Métricas

In [19]:
print("📊 EXTRAYENDO MÉTRICAS DE LOS MODELOS...")
print("="*80)

# DataFrame para modelos de clasificación
classification_metrics = []

for name, model_info in models.items():
    if model_info['type'] == 'clasificacion' and model_info['package'] is not None:
        metrics = model_info['package'].get('metrics', {})
        classification_metrics.append({
            'Modelo': name,
            'Accuracy_Train': metrics.get('accuracy_train', np.nan),
            'Accuracy_Test': metrics.get('accuracy_test', np.nan),
            'Precision': metrics.get('precision', np.nan),
            'Recall': metrics.get('recall', np.nan),
            'F1_Score': metrics.get('f1_score', np.nan),
            'CV_Accuracy': metrics.get('cv_accuracy_mean', np.nan)
        })

df_classification = pd.DataFrame(classification_metrics)

print("\n📋 MÉTRICAS DE MODELOS DE CLASIFICACIÓN:")
print("="*80)
if len(df_classification) > 0:
    display(df_classification.style.format({
        'Accuracy_Train': '{:.4f}',
        'Accuracy_Test': '{:.4f}',
        'Precision': '{:.4f}',
        'Recall': '{:.4f}',
        'F1_Score': '{:.4f}',
        'CV_Accuracy': '{:.4f}'
    }).background_gradient(cmap='RdYlGn', subset=['Accuracy_Test', 'F1_Score']))
else:
    print("   ⚠️ No hay modelos de clasificación cargados")

📊 EXTRAYENDO MÉTRICAS DE LOS MODELOS...

📋 MÉTRICAS DE MODELOS DE CLASIFICACIÓN:
   ⚠️ No hay modelos de clasificación cargados


In [20]:
# Métricas de regresión
regression_metrics = []

for name, model_info in models.items():
    if model_info['type'] == 'regresion' and model_info['package'] is not None:
        metrics = model_info['package'].get('metrics', {})
        regression_metrics.append({
            'Modelo': name,
            'R2_Train': metrics.get('r2_train', np.nan),
            'R2_Test': metrics.get('r2_test', np.nan),
            'RMSE_Train': metrics.get('rmse_train', np.nan),
            'RMSE_Test': metrics.get('rmse_test', np.nan),
            'MAE_Test': metrics.get('mae_test', np.nan),
            'CV_R2': metrics.get('cv_r2_mean', np.nan)
        })

df_regression = pd.DataFrame(regression_metrics)

print("\n📋 MÉTRICAS DE MODELOS DE REGRESIÓN:")
print("="*80)
if len(df_regression) > 0:
    display(df_regression.style.format({
        'R2_Train': '{:.4f}',
        'R2_Test': '{:.4f}',
        'RMSE_Train': '${:,.2f}',
        'RMSE_Test': '${:,.2f}',
        'MAE_Test': '${:,.2f}',
        'CV_R2': '{:.4f}'
    }).background_gradient(cmap='RdYlGn', subset=['R2_Test']))
else:
    print("   ⚠️ No hay modelos de regresión cargados")


📋 MÉTRICAS DE MODELOS DE REGRESIÓN:
   ⚠️ No hay modelos de regresión cargados


In [21]:
# Métricas de clustering
clustering_metrics = []

for name, model_info in models.items():
    if model_info['type'] == 'clustering' and model_info['package'] is not None:
        metrics = model_info['package'].get('metrics', {})
        clustering_metrics.append({
            'Modelo': name,
            'Silhouette': metrics.get('silhouette', np.nan),
            'Davies_Bouldin': metrics.get('davies_bouldin', np.nan),
            'Inercia': metrics.get('inertia', np.nan),
            'N_Clusters': model_info['package'].get('n_clusters', np.nan)
        })

df_clustering = pd.DataFrame(clustering_metrics)

print("\n📋 MÉTRICAS DE MODELOS DE CLUSTERING:")
print("="*80)
if len(df_clustering) > 0:
    display(df_clustering.style.format({
        'Silhouette': '{:.4f}',
        'Davies_Bouldin': '{:.4f}',
        'Inercia': '{:,.2f}',
        'N_Clusters': '{:.0f}'
    }))
else:
    print("   ⚠️ No hay modelos de clustering cargados")


📋 MÉTRICAS DE MODELOS DE CLUSTERING:
   ⚠️ No hay modelos de clustering cargados


## 4. Visualizaciones Comparativas

In [22]:
# Gráfico comparativo de modelos de clasificación
if len(df_classification) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Accuracy
    ax = axes[0, 0]
    x = np.arange(len(df_classification))
    width = 0.35
    ax.bar(x - width/2, df_classification['Accuracy_Train'], width, label='Train', alpha=0.8)
    ax.bar(x + width/2, df_classification['Accuracy_Test'], width, label='Test', alpha=0.8)
    ax.set_ylabel('Accuracy', fontsize=11)
    ax.set_title('Comparación de Accuracy', fontsize=13, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(df_classification['Modelo'], rotation=45, ha='right')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    # F1-Score
    ax = axes[0, 1]
    ax.bar(df_classification['Modelo'], df_classification['F1_Score'], color='coral')
    ax.set_ylabel('F1-Score', fontsize=11)
    ax.set_title('Comparación de F1-Score', fontsize=13, fontweight='bold')
    ax.set_xticklabels(df_classification['Modelo'], rotation=45, ha='right')
    ax.grid(axis='y', alpha=0.3)
    
    # Precision vs Recall
    ax = axes[1, 0]
    x = np.arange(len(df_classification))
    ax.bar(x - width/2, df_classification['Precision'], width, label='Precision', alpha=0.8)
    ax.bar(x + width/2, df_classification['Recall'], width, label='Recall', alpha=0.8)
    ax.set_ylabel('Score', fontsize=11)
    ax.set_title('Precision vs Recall', fontsize=13, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(df_classification['Modelo'], rotation=45, ha='right')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    # Validación Cruzada
    ax = axes[1, 1]
    ax.bar(df_classification['Modelo'], df_classification['CV_Accuracy'], color='lightgreen')
    ax.set_ylabel('CV Accuracy', fontsize=11)
    ax.set_title('Validación Cruzada (5-Fold)', fontsize=13, fontweight='bold')
    ax.set_xticklabels(df_classification['Modelo'], rotation=45, ha='right')
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Gráficos comparativos generados para modelos de clasificación")
else:
    print("⚠️ No hay suficientes modelos de clasificación para comparar")

⚠️ No hay suficientes modelos de clasificación para comparar


In [23]:
# Radar chart para comparación integral
if len(df_classification) > 0:
    from math import pi
    
    # Preparar datos para radar chart
    categories = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    N = len(categories)
    
    # Crear figura
    fig = plt.figure(figsize=(14, 10))
    ax = fig.add_subplot(111, projection='polar')
    
    # Ángulos para cada métrica
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]
    
    # Colores para cada modelo
    colors = plt.cm.Set2(np.linspace(0, 1, len(df_classification)))
    
    # Plotear cada modelo
    for idx, row in df_classification.iterrows():
        values = [
            row['Accuracy_Test'],
            row['Precision'],
            row['Recall'],
            row['F1_Score']
        ]
        values += values[:1]
        
        ax.plot(angles, values, 'o-', linewidth=2, label=row['Modelo'], color=colors[idx])
        ax.fill(angles, values, alpha=0.15, color=colors[idx])
    
    # Configuración
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, size=12)
    ax.set_ylim(0, 1)
    ax.set_title('Comparación Integral de Modelos de Clasificación\n(Radar Chart)', 
                size=16, fontweight='bold', pad=20)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    ax.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Radar Chart generado exitosamente")
else:
    print("⚠️ No hay suficientes modelos de clasificación para radar chart")

⚠️ No hay suficientes modelos de clasificación para radar chart


## 5. Ranking y Selección del Mejor Modelo

In [24]:
print("🏆 RANKING DE MODELOS")
print("="*80)

# Ranking para modelos de clasificación
if len(df_classification) > 0:
    print("\n📊 MODELOS DE CLASIFICACIÓN:")
    print("-" * 80)
    
    # Calcular score compuesto (promedio ponderado)
    df_classification['Score_Compuesto'] = (
        0.30 * df_classification['Accuracy_Test'] +
        0.25 * df_classification['F1_Score'] +
        0.20 * df_classification['Precision'] +
        0.15 * df_classification['Recall'] +
        0.10 * df_classification['CV_Accuracy']
    )
    
    # Ordenar por score compuesto
    df_classification_ranked = df_classification.sort_values('Score_Compuesto', ascending=False)
    df_classification_ranked['Rank'] = range(1, len(df_classification_ranked) + 1)
    
    display(df_classification_ranked[['Rank', 'Modelo', 'Accuracy_Test', 'F1_Score', 
                                      'Precision', 'Recall', 'Score_Compuesto']].style.format({
        'Accuracy_Test': '{:.4f}',
        'F1_Score': '{:.4f}',
        'Precision': '{:.4f}',
        'Recall': '{:.4f}',
        'Score_Compuesto': '{:.4f}'
    }).background_gradient(cmap='RdYlGn', subset=['Score_Compuesto']))
    
    # Mejor modelo
    best_classification = df_classification_ranked.iloc[0]
    print(f"\n🥇 MEJOR MODELO DE CLASIFICACIÓN: {best_classification['Modelo']}")
    print(f"   • Accuracy Test: {best_classification['Accuracy_Test']:.4f}")
    print(f"   • F1-Score: {best_classification['F1_Score']:.4f}")
    print(f"   • Score Compuesto: {best_classification['Score_Compuesto']:.4f}")
else:
    print("\n⚠️ No hay modelos de clasificación para rankear")
    best_classification = None

🏆 RANKING DE MODELOS

⚠️ No hay modelos de clasificación para rankear


In [25]:
# Ranking para modelos de regresión
if len(df_regression) > 0:
    print("\n📊 MODELOS DE REGRESIÓN:")
    print("-" * 80)
    
    # Para regresión, R² más alto es mejor
    df_regression_ranked = df_regression.sort_values('R2_Test', ascending=False)
    df_regression_ranked['Rank'] = range(1, len(df_regression_ranked) + 1)
    
    display(df_regression_ranked[['Rank', 'Modelo', 'R2_Test', 'RMSE_Test', 'MAE_Test']].style.format({
        'R2_Test': '{:.4f}',
        'RMSE_Test': '${:,.2f}',
        'MAE_Test': '${:,.2f}'
    }).background_gradient(cmap='RdYlGn', subset=['R2_Test']))
    
    # Mejor modelo
    best_regression = df_regression_ranked.iloc[0]
    print(f"\n🥇 MEJOR MODELO DE REGRESIÓN: {best_regression['Modelo']}")
    print(f"   • R² Test: {best_regression['R2_Test']:.4f}")
    print(f"   • RMSE Test: ${best_regression['RMSE_Test']:,.2f}")
    print(f"   • MAE Test: ${best_regression['MAE_Test']:,.2f}")
else:
    print("\n⚠️ No hay modelos de regresión para rankear")
    best_regression = None


⚠️ No hay modelos de regresión para rankear


In [26]:
# Información de clustering
if len(df_clustering) > 0:
    print("\n📊 MODELOS DE CLUSTERING:")
    print("-" * 80)
    
    display(df_clustering.style.format({
        'Silhouette': '{:.4f}',
        'Davies_Bouldin': '{:.4f}',
        'Inercia': '{:,.2f}',
        'N_Clusters': '{:.0f}'
    }))
    
    best_clustering = df_clustering.iloc[0]
    print(f"\n📍 MODELO DE CLUSTERING: {best_clustering['Modelo']}")
    print(f"   • Silhouette Score: {best_clustering['Silhouette']:.4f}")
    print(f"   • Davies-Bouldin Index: {best_clustering['Davies_Bouldin']:.4f}")
    print(f"   • Número de Clusters: {best_clustering['N_Clusters']:.0f}")
else:
    print("\n⚠️ No hay modelos de clustering disponibles")
    best_clustering = None


⚠️ No hay modelos de clustering disponibles



## 6. Análisis de Fortalezas y Debilidades

In [27]:
print("\n" + "="*80)
print("📊 ANÁLISIS COMPARATIVO DE MODELOS")
print("="*80)

if len(df_classification) > 0:
    print("\n🔍 ANÁLISIS DE CLASIFICACIÓN:")
    print("-" * 80)
    
    # Modelo con mejor accuracy
    best_acc = df_classification.loc[df_classification['Accuracy_Test'].idxmax()]
    print(f"\n✨ Mejor Accuracy: {best_acc['Modelo']} ({best_acc['Accuracy_Test']:.4f})")
    
    # Modelo con mejor F1
    best_f1 = df_classification.loc[df_classification['F1_Score'].idxmax()]
    print(f"✨ Mejor F1-Score: {best_f1['Modelo']} ({best_f1['F1_Score']:.4f})")
    
    # Modelo con mejor balance precision-recall
    df_classification['Balance_PR'] = df_classification['Precision'] * df_classification['Recall']
    best_balance = df_classification.loc[df_classification['Balance_PR'].idxmax()]
    print(f"✨ Mejor Balance Precision-Recall: {best_balance['Modelo']}")
    
    # Modelo más estable (menor diferencia train-test)
    df_classification['Estabilidad'] = abs(df_classification['Accuracy_Train'] - df_classification['Accuracy_Test'])
    most_stable = df_classification.loc[df_classification['Estabilidad'].idxmin()]
    print(f"✨ Modelo Más Estable: {most_stable['Modelo']} (diferencia: {most_stable['Estabilidad']:.4f})")
    
    # Análisis de overfitting
    print("\n⚠️ DETECCIÓN DE OVERFITTING:")
    for idx, row in df_classification.iterrows():
        diff = row['Accuracy_Train'] - row['Accuracy_Test']
        if diff > 0.1:
            print(f"   • {row['Modelo']}: Posible overfitting (diferencia: {diff:.4f})")
        elif diff < -0.05:
            print(f"   • {row['Modelo']}: Comportamiento inusual (test > train)")
        else:
            print(f"   • {row['Modelo']}: Buen balance (diferencia: {diff:.4f})")


📊 ANÁLISIS COMPARATIVO DE MODELOS


## 7. Recomendación Final

In [28]:
print("\n" + "="*80)
print("🏆 RECOMENDACIÓN FINAL - MEJOR MODELO PARA PRODUCCIÓN")
print("="*80)

if best_classification is not None:
    print(f"\n🎯 MODELO SELECCIONADO: {best_classification['Modelo']}")
    print("-" * 80)
    
    print(f"\n📊 MÉTRICAS DE RENDIMIENTO:")
    print(f"   • Accuracy (Test): {best_classification['Accuracy_Test']:.4f}")
    print(f"   • F1-Score: {best_classification['F1_Score']:.4f}")
    print(f"   • Precision: {best_classification['Precision']:.4f}")
    print(f"   • Recall: {best_classification['Recall']:.4f}")
    print(f"   • CV Accuracy: {best_classification['CV_Accuracy']:.4f}")
    print(f"   • Score Compuesto: {best_classification['Score_Compuesto']:.4f}")
    
    print(f"\n✅ FORTALEZAS:")
    if best_classification['Accuracy_Test'] > 0.8:
        print(f"   • Excelente precisión general (Accuracy > 0.8)")
    if best_classification['F1_Score'] > 0.75:
        print(f"   • Buen balance entre Precision y Recall")
    if abs(best_classification['Accuracy_Train'] - best_classification['Accuracy_Test']) < 0.05:
        print(f"   • Modelo estable sin overfitting significativo")
    
    print(f"\n💡 RAZONES DE SELECCIÓN:")
    print(f"   1. Mayor score compuesto entre todos los modelos")
    print(f"   2. Balance óptimo entre todas las métricas")
    print(f"   3. Validación cruzada consistente")
    print(f"   4. Adecuado para despliegue en producción")
    
    print(f"\n🚀 SIGUIENTE PASO:")
    print(f"   Implementar {best_classification['Modelo']} en aplicación FullStack")

elif best_regression is not None:
    print(f"\n🎯 MODELO SELECCIONADO: {best_regression['Modelo']}")
    print("-" * 80)
    print(f"\n📊 MÉTRICAS DE RENDIMIENTO:")
    print(f"   • R² (Test): {best_regression['R2_Test']:.4f}")
    print(f"   • RMSE (Test): ${best_regression['RMSE_Test']:,.2f}")
    print(f"   • MAE (Test): ${best_regression['MAE_Test']:,.2f}")
else:
    print("\n⚠️ No hay modelos suficientes para hacer una recomendación")


🏆 RECOMENDACIÓN FINAL - MEJOR MODELO PARA PRODUCCIÓN

⚠️ No hay modelos suficientes para hacer una recomendación


## 8. Guardar Resultados de la Comparación

In [29]:
# Guardar resultados
print("💾 GUARDANDO RESULTADOS...")
print("="*80)

if len(df_classification) > 0:
    df_classification_ranked.to_csv('model_comparison_classification.csv', index=False)
    print("   ✓ model_comparison_classification.csv")

if len(df_regression) > 0:
    df_regression_ranked.to_csv('model_comparison_regression.csv', index=False)
    print("   ✓ model_comparison_regression.csv")

if len(df_clustering) > 0:
    df_clustering.to_csv('model_comparison_clustering.csv', index=False)
    print("   ✓ model_comparison_clustering.csv")

# Guardar información del mejor modelo
best_model_info = {
    'classification': best_classification.to_dict() if best_classification is not None else None,
    'regression': best_regression.to_dict() if best_regression is not None else None,
    'clustering': best_clustering.to_dict() if best_clustering is not None else None
}

with open('best_model_selection.pkl', 'wb') as f:
    pickle.dump(best_model_info, f)
print("   ✓ best_model_selection.pkl")

print("\n✅ Todos los resultados guardados exitosamente")

💾 GUARDANDO RESULTADOS...
   ✓ best_model_selection.pkl

✅ Todos los resultados guardados exitosamente


## 9. Resumen Ejecutivo

In [30]:
print("\n" + "="*80)
print("📋 RESUMEN EJECUTIVO - COMPARACIÓN DE MODELOS ML")
print("="*80)

print(f"\n📊 MODELOS EVALUADOS:")
print(f"   • Clasificación: {len(df_classification)} modelos")
print(f"   • Regresión: {len(df_regression)} modelos")
print(f"   • Clustering: {len(df_clustering)} modelos")
print(f"   • TOTAL: {len(df_classification) + len(df_regression) + len(df_clustering)} modelos")

if best_classification is not None:
    print(f"\n🥇 GANADOR (Clasificación): {best_classification['Modelo']}")
    print(f"   • Accuracy: {best_classification['Accuracy_Test']:.4f}")
    print(f"   • F1-Score: {best_classification['F1_Score']:.4f}")

if best_regression is not None:
    print(f"\n🥇 GANADOR (Regresión): {best_regression['Modelo']}")
    print(f"   • R²: {best_regression['R2_Test']:.4f}")
    print(f"   • RMSE: ${best_regression['RMSE_Test']:,.2f}")

print(f"\n📁 ARCHIVOS GENERADOS:")
print(f"   • model_comparison_classification.csv")
print(f"   • model_comparison_regression.csv")
print(f"   • model_comparison_clustering.csv")
print(f"   • best_model_selection.pkl")

print(f"\n🚀 PRÓXIMO PASO:")
print(f"   Desarrollar aplicación FullStack con el modelo seleccionado")

print("\n" + "="*80)
print("✅ ANÁLISIS COMPARATIVO COMPLETADO")
print("="*80)


📋 RESUMEN EJECUTIVO - COMPARACIÓN DE MODELOS ML

📊 MODELOS EVALUADOS:
   • Clasificación: 0 modelos
   • Regresión: 0 modelos
   • Clustering: 0 modelos
   • TOTAL: 0 modelos

📁 ARCHIVOS GENERADOS:
   • model_comparison_classification.csv
   • model_comparison_regression.csv
   • model_comparison_clustering.csv
   • best_model_selection.pkl

🚀 PRÓXIMO PASO:
   Desarrollar aplicación FullStack con el modelo seleccionado

✅ ANÁLISIS COMPARATIVO COMPLETADO
